# 2. Extracting Token Transfers
In this excercise, we will be extracting token transfers from a single token contract.
We will obtain these transfers from the event/log data, using web3py.

As in the previous excercise, please use your own endpoint URL. You can create a free account with [Moralis.io](https://admin.moralis.io/register), and get an endpoint URL from them. You may be able to use this notebook endpoint, but you may run into rate limits if other participants are using it at the same time. Or at a later point in time this endpoint may not work at all anymore.

In [1]:
endpoint = "https://speedy-nodes-nyc.moralis.io/03c966587b022c980f59136b/eth/mainnet/archive"

In [2]:
from web3 import Web3
w3 = Web3(Web3.HTTPProvider(endpoint))
w3.isConnected()

True

# 2.1 Transfer Event ABI

In [4]:
# Reduced ERC-20 ABI, only Transfer event
import json
ABIstring = """[
    {
        "anonymous": false,
        "inputs": [
            {
                "indexed": true,
                "name": "from",
                "type": "address"
            },
            {
                "indexed": true,
                "name": "to",
                "type": "address"
            },
            {
                "indexed": false,
                "name": "value",
                "type": "uint256"
            }
        ],
        "name": "Transfer",
        "type": "event"
    }
]
"""
anonERC20contract = w3.eth.contract(abi=json.loads(ABIstring))
transferEventType = anonERC20contract.events.Transfer
transferEventABI = transferEventType._get_event_abi()

In [5]:
tokenContractAddress = "0xef51c9377feb29856e61625caf9390bd0b67ea18" # Bionic token contract address

In [14]:
from tqdm import tqdm
import itertools

logLists = []
blockStart = 6018000
blockEnd = blockStart + 100000

for blockNumber in tqdm(range(blockStart, blockEnd, 2000)):
    logs = w3.eth.get_logs({"fromBlock": str(hex(blockNumber)),
                            "toBlock": str(hex(blockNumber+2000)), #6018000
                            "address": Web3.toChecksumAddress(tokenContractAddress),
                            "topics": [Web3.keccak(text='Transfer(address,address,uint256)').hex()]})
    logLists.append(logs)
logs = list(itertools.chain.from_iterable(logLists))

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:14<00:00,  3.53it/s]


In [16]:
from web3._utils.events import get_event_data

transferEvents = []
for log in logs:
        log = dict(get_event_data(w3.codec, transferEventABI, log))
        log["transactionHash"] = log["transactionHash"].hex()
        del log["blockHash"]
        for k,v in log["args"].items():
            log[k] = v
        del log["args"]
        transferEvents.append(log)

In [17]:
import pandas as pd
tokenTransfersDF = pd.DataFrame(transferEvents)
tokenTransfersDF

,event,logIndex,transactionIndex,transactionHash,address,blockNumber,from,to,value
0,Transfer,7,22,0x859bdc6ace1a2c7d1c7d612c95535b89b8986a8f4742...,0xEf51c9377FeB29856E61625cAf9390bD0B67eA18,6018000,0xdD2A5B646bb936CbC279CBE462E31eab2C309452,0x7bC63d25e3fCA40FFc33055dd30810e09bfC7aFF,2500000000000
1,Transfer,8,23,0x0566fef48edfd42ea0593626d92e54e2e99c4c085c2d...,0xEf51c9377FeB29856E61625cAf9390bD0B67eA18,6018000,0xdD2A5B646bb936CbC279CBE462E31eab2C309452,0x1d2CeA478a53fBCc48957a752F83b09e6152A5F7,2500000000000
2,Transfer,192,97,0xeba05dd5582ec2cd77550a83608d43153ac07ed2f766...,0xEf51c9377FeB29856E61625cAf9390bD0B67eA18,6018001,0xdD2A5B646bb936CbC279CBE462E31eab2C309452,0x6c19B6FA36324774bCd22eB56722Fc576E32dCC9,2500000000000
3,Transfer,140,54,0x41960bc310ccd12e43f23a009e3641db70d3f762d906...,0xEf51c9377FeB29856E61625cAf9390bD0B67eA18,6018003,0xdD2A5B646bb936CbC279CBE462E31eab2C309452,0x37084411b0cbCb974BEd6111bE0ECE0C82617d4A,2500000000000
4,Transfer,6,15,0xef56622712708e55358b0065b7b64581e984502aa5be...,0xEf51c9377FeB29856E61625cAf9390bD0B67eA18,6018004,0xdD2A5B646bb936CbC279CBE462E31eab2C309452,0x3cF12f7871a90E84Eb44767f06c5E7d828b27897,2500000000000
...,...,...,...,...,...,...,...,...,...
10669,Transfer,40,45,0xba9fdf92a995b3622c60189480406dc2bb0fa08c5df8...,0xEf51c9377FeB29856E61625cAf9390bD0B67eA18,6113383,0x0a7C759c68dF05d622d8969a564B002787D9A778,0x2a0c0DBEcC7E4D658f48E01e3fA353F44050c208,2500000000000
10670,Transfer,28,45,0xe97582852973e48c7fa1d267d88dd82abc24681e7d66...,0xEf51c9377FeB29856E61625cAf9390bD0B67eA18,6113750,0x1b52b1A10F3B81B3cc2ed7fFCB00419eea68F79e,0x41e7e5b8B3c04Cfd2caCE661AF26E4A54Cf6220E,2500000000000
10671,Transfer,0,0,0x1b5afe3fb0a7dd83bb89c0e107d55596bddce8868cea...,0xEf51c9377FeB29856E61625cAf9390bD0B67eA18,6116179,0x1B87C2a6058BC88548Bc9bB18b0717f939B2CCB3,0x2a0c0DBEcC7E4D658f48E01e3fA353F44050c208,2500000000000
10672,Transfer,9,57,0x9a2529fdcbe5a65c6ed0a1643fbad1f1604ff4ec75bb...,0xEf51c9377FeB29856E61625cAf9390bD0B67eA18,6116326,0x1cB51f2ccc7BEB2419d62Ea3a84BD66f3B9A444C,0x2a0c0DBEcC7E4D658f48E01e3fA353F44050c208,39919940737580


In [21]:
import networkx as nx

In [22]:
G = nx.from_pandas_edgelist(df=tokenTransfersDF, source="from", target="to", create_using=nx.DiGraph)

In [31]:
df = pd.DataFrame(dict(
    indegree = dict(G.in_degree),
    outdegree = dict(G.out_degree),
    indegree_centrality = nx.in_degree_centrality(G)
))

In [32]:
df.sort_values("indegree")

,indegree,outdegree,indegree_centrality
0xdD2A5B646bb936CbC279CBE462E31eab2C309452,0,9734,0.000000
0xB808Ea87da53fd1A9B064B5fbE647a727c7376C6,0,1,0.000000
0xf22640A3d0351782D35d6F206fEdA9Cafd7068cC,0,1,0.000000
0x3B3F9F751328Bb323361D443887597699d9f0CE5,0,1,0.000000
0x858aC1DF0F5E96Fa0e94bD44A1cD601A339a6E3d,0,1,0.000000
...,...,...,...
0x03cCFEA2034f1B7dF7eAd723d795F846d9d5a81E,34,2,0.003460
0x2a0c0DBEcC7E4D658f48E01e3fA353F44050c208,35,4,0.003562
0x8d12A197cB00D4747a1fe03395095ce2A5CC6819,39,11,0.003969
0x74B9f1d9F994d561c238db9261De93eE95362889,42,1,0.004274


In [18]:
from pyvis.network import Network

In [19]:
transferNetwork = Network(height='750px', width='100%', notebook=True)
# set the physics layout of the network
transferNetwork.barnes_hut()

sampleSize = 500
for row in tokenTransfersDF.sample(sampleSize)[["from","to"]].to_dict('records'):
    transferNetwork.add_node(row["from"], row["from"], title=row["from"])
    transferNetwork.add_node(row["to"], row["to"], title=row["to"])
    transferNetwork.add_edge(row["from"], row["to"], value=1)

In [20]:
transferNetwork.show("tokenTransferGraph.html")

In [13]:
0xef51c9377feb29856e61625caf9390bd0b67ea18 # bionic

1366272683043014078604117687716855698063859640856